In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib as mpl
# import japanize_matplotlib
# mpl.rcParams['font.family'] = 'MS Gothic"
import seaborn as sns
import lightgbm as lgb

from datetime import date,timedelta
from dateutil import relativedelta
# from sklearn.metrics import mean_wquared_error

from sklearn.preprocessing import LabelEncoder

In [48]:
train_df = pd.read_csv('data/preprocessed_train.csv')
test_df = pd.read_csv('data/preprocessed_test.csv')
weather_df = pd.read_csv('data/preprocessed_weather.csv')

In [3]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

kind            0
date            0
amount          0
mode_price      0
area            0
mean_temp     212
max_temp      212
min_temp      213
sum_rain      212
sun_time      260
mean_humid    232
dtype: int64
kind            0
date            0
amount        320
mode_price    320
area            0
mean_temp     320
max_temp      320
min_temp      320
sum_rain      320
sun_time      320
mean_humid    320
dtype: int64


In [49]:
train_df

,kind,date,amount,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid
0,だいこん,2005-11-07,201445.0,735.0,千葉,18.800000,24.400000,14.700000,7.000000,5.500000,64.00000
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森,13.891667,18.944792,9.200000,6.156250,6.641667,58.65625
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森,11.723958,16.320833,7.609375,4.067708,6.051042,64.03125
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森,11.150000,14.150000,8.100000,0.750000,0.200000,76.50000
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森,10.900000,14.950000,8.900000,5.250000,2.900000,61.00000
...,...,...,...,...,...,...,...,...,...,...,...
97777,しめじ,2017-05-26,17231.0,48.5,新潟_長野,18.650000,24.050000,14.650000,10.000000,1.450000,79.00000
97778,しめじ,2017-05-27,17105.0,48.5,新潟_長野,16.650000,21.400000,13.300000,2.000000,6.850000,75.00000
97779,しめじ,2017-05-29,12406.0,48.5,新潟_長野,19.800000,28.200000,10.900000,0.000000,12.750000,57.00000
97780,しめじ,2017-05-30,12320.0,48.5,新潟_長野,22.350000,29.850000,15.350000,0.000000,10.000000,60.00000


In [50]:
test_df

,kind,date,amount,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid
0,だいこん,2022-05-02,NaN,NaN,千葉_全国_青森,NaN,NaN,NaN,NaN,NaN,NaN
1,だいこん,2022-05-06,NaN,NaN,千葉_全国_青森,NaN,NaN,NaN,NaN,NaN,NaN
2,だいこん,2022-05-07,NaN,NaN,千葉_全国_青森,NaN,NaN,NaN,NaN,NaN,NaN
3,だいこん,2022-05-09,NaN,NaN,千葉_全国_青森,NaN,NaN,NaN,NaN,NaN,NaN
4,だいこん,2022-05-10,NaN,NaN,千葉_全国_青森,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
315,ミニトマト,2022-05-26,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
316,ミニトマト,2022-05-27,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
317,ミニトマト,2022-05-28,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
318,ミニトマト,2022-05-30,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
params = {
    'objective':'regression',
    'metric':'rmse',
    'num_leaves':42,
    'max_depth':7,
    'feature_fraction':0.95,
    'subsample_freq':1,
    'bagging_fraction':0.95,
    'min_data_in_leaf':2,
    'learning_rate':0.1,
    'boosting':'gbdt',
    'lambda_l1':0.1,
    'lambda_l2':10,
    'verbosity':-1,
    'random_state':42,
    'num_boost_round':50000,
    'early_stopping_rounds':100,
}

In [5]:
all_df = pd.concat([train_df, test_df])

In [6]:
all_df

,kind,date,amount,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid
0,だいこん,2005-11-07,201445.0,735.0,千葉,18.800000,24.400000,14.700000,7.000000,5.500000,64.00000
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森,13.891667,18.944792,9.200000,6.156250,6.641667,58.65625
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森,11.723958,16.320833,7.609375,4.067708,6.051042,64.03125
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森,11.150000,14.150000,8.100000,0.750000,0.200000,76.50000
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森,10.900000,14.950000,8.900000,5.250000,2.900000,61.00000
...,...,...,...,...,...,...,...,...,...,...,...
315,ミニトマト,2022-05-26,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
316,ミニトマト,2022-05-27,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
317,ミニトマト,2022-05-28,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
318,ミニトマト,2022-05-30,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
test_df.date.max()
test_df.date.min()

'2022-05-02'

In [8]:
all_df

,kind,date,amount,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid
0,だいこん,2005-11-07,201445.0,735.0,千葉,18.800000,24.400000,14.700000,7.000000,5.500000,64.00000
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森,13.891667,18.944792,9.200000,6.156250,6.641667,58.65625
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森,11.723958,16.320833,7.609375,4.067708,6.051042,64.03125
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森,11.150000,14.150000,8.100000,0.750000,0.200000,76.50000
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森,10.900000,14.950000,8.900000,5.250000,2.900000,61.00000
...,...,...,...,...,...,...,...,...,...,...,...
315,ミニトマト,2022-05-26,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
316,ミニトマト,2022-05-27,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
317,ミニトマト,2022-05-28,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
318,ミニトマト,2022-05-30,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# バリデーションはHold-outa法
val_period = (pd.to_datetime(train_df.date.max()) - relativedelta.relativedelta(years=1)).strftime("%Y-%m-%d") # 直近1年間を検証期間とする

tr_df = train_df.query('date < @val_period')
va_df = train_df.query('@val_period <= date')

In [11]:
# lightgbmはカテゴリ変数に対応していないのでintに変換
# date : yearとmonthとdayに分割
def make_year_month_day(df):
    year = pd.to_datetime(df.date).dt.strftime("%Y").astype('int16')
    month = pd.to_datetime(df.date).dt.strftime("%m").astype('int16')
    day = pd.to_datetime(df.date).dt.strftime("%d").astype('int16')
    return year, month, day

tr_df['year'], tr_df['month'], tr_df['day'] = make_year_month_day(tr_df)
va_df['year'], va_df['month'], va_df['day'] = make_year_month_day(va_df)
test_df['year'], test_df['month'], test_df['day'] = make_year_month_day(test_df)
tr_df = tr_df.drop("date", axis=1)
va_df = va_df.drop("date", axis=1)
test_df = test_df.drop("date", axis=1)

# area : 一旦int型にカテゴリ変数を変換し、light gbmのオプションcategorical_featureで指定。そのほうがonehotよりも良いらしい
# 他のエンコーディングは後で試してみる（onehotエンコーディング、targetエンコーディング）
area_le = LabelEncoder()
tr_df['area'] = area_le.fit_transform(tr_df['area'])
va_df['area'] = area_le.fit_transform(va_df['area'])
test_df['area'] = area_le.fit_transform(test_df['area'])

# kind : 同じくcategorical_featureで指定するためint型に変換
kind_le = LabelEncoder()
tr_df['kind'] = kind_le.fit_transform(tr_df['kind'])
va_df['kind'] = kind_le.fit_transform(va_df['kind'])
test_df['kind'] = kind_le.fit_transform(test_df['kind'])

cat_cols = ['area', 'kind']

C:\Users\owner\AppData\Local\Temp\ipykernel_5908\3581775403.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['year'], tr_df['month'], tr_df['day'] = make_year_month_day(tr_df)
C:\Users\owner\AppData\Local\Temp\ipykernel_5908\3581775403.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  va_df['year'], va_df['month'], va_df['day'] = make_year_month_day(va_df)


In [12]:
print(tr_df.shape)
print(va_df.shape)
print(train_df.shape)

(92095, 13)
(5687, 13)
(97782, 11)


In [13]:
tr_x, tr_y = tr_df.drop("mode_price",axis=1), tr_df["mode_price"]
va_x, va_y = va_df.drop("mode_price",axis=1), va_df["mode_price"]
test_x, test_y = test_df.drop("mode_price",axis=1), test_df["mode_price"]

tr_data = lgb.Dataset(tr_x, label=tr_y)
va_data = lgb.Dataset(va_x, label=va_y)

model = lgb.train(
    params,
    tr_data,
    categorical_feature = cat_cols,
    valid_names = ['train', 'valid'],
    valid_sets = [tr_data, va_data],
    verbose_eval = 10,
)

Training until validation scores don't improve for 100 rounds
[10]	train's rmse: 666.325	valid's rmse: 2106.63
[20]	train's rmse: 507.336	valid's rmse: 2376.93
[30]	train's rmse: 458.166	valid's rmse: 2653.4
[40]	train's rmse: 435.102	valid's rmse: 2810.79
[50]	train's rmse: 418.463	valid's rmse: 2880.87
[60]	train's rmse: 405.945	valid's rmse: 2910.44
[70]	train's rmse: 396.876	valid's rmse: 2918
[80]	train's rmse: 389.345	valid's rmse: 2922.37
[90]	train's rmse: 383.433	valid's rmse: 2921.34
[100]	train's rmse: 378.185	valid's rmse: 2923.18
Early stopping, best iteration is:
[2]	train's rmse: 1092.03	valid's rmse: 1683.23


c:\Users\owner\anaconda3\envs\ds\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\owner\anaconda3\envs\ds\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['area', 'kind']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\owner\anaconda3\envs\ds\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\owner\anaconda3\envs\ds\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the 

In [14]:
val_pred = model.predict(va_x, num_iteration=model.best_iteration)

pred_df = pd.DataFrame(sorted(zip(va_x.index, val_pred, va_y)), columns=['index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(), tr_x.columns)), columns=['importance', 'feature'])

test_pred = model.predict(test_x, num_iteration=model.best_iteration)


In [43]:
result = test_x[['kind', 'year', 'month', 'day']]
result['mode_price'] = test_pred

result['date'] = result['year'].astype('str') + result['month'].astype('str').str.zfill(2) + result['day'].astype('str').str.zfill(2)

result['kind'] = kind_le.inverse_transform(result.kind)

result = result[['kind', 'date', 'mode_price']]

result

C:\Users\owner\AppData\Local\Temp\ipykernel_5908\1024664812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['mode_price'] = test_pred
C:\Users\owner\AppData\Local\Temp\ipykernel_5908\1024664812.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['date'] = result['year'].astype('str') + result['month'].astype('str').str.zfill(2) + result['day'].astype('str').str.zfill(2)
C:\Users\owner\AppData\Local\Temp\ipykernel_5908\1024664812.py:6: SettingWithCopyWarning: 
A value is trying to be set on

,kind,date,mode_price
0,だいこん,20220502,1200.8998
1,だいこん,20220506,1200.8998
2,だいこん,20220507,1200.8998
3,だいこん,20220509,1200.8998
4,だいこん,20220510,1200.8998
...,...,...,...
315,ミニトマト,20220526,1200.8998
316,ミニトマト,20220527,1200.8998
317,ミニトマト,20220528,1200.8998
318,ミニトマト,20220530,1200.8998


In [47]:
result.to_csv('data/submission1.csv', index=False)

In [18]:
sub_df = pd.read_csv('data/sample_submission.csv')

In [19]:
sub_df

,kind,date,mode_price
0,だいこん,20220502,0
1,だいこん,20220506,0
2,だいこん,20220507,0
3,だいこん,20220509,0
4,だいこん,20220510,0
...,...,...,...
315,ミニトマト,20220526,0
316,ミニトマト,20220527,0
317,ミニトマト,20220528,0
318,ミニトマト,20220530,0


In [14]:
kinds = all_df.kind.unique()

In [10]:
for kind in kinds:
    all_df = pd.concat([train_df, test_df])
    tmp_tra_df = all_df[all_df['kind'] == kind]

    

    tra_df = train_df.iloc[-100:]
    va_df = train_df.iloc[:-100]

In [11]:
tr_df = train_df
va_df = 

tr_x, tr_y =
va_x, va_y

SyntaxError: invalid syntax (2343256968.py, line 2)

In [22]:
model = lgb.Dataset()

(320, 12)